In [1]:
# Code for analyzing NFL big data bowl data (probably with Celia!!!)

In [13]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from pathlib import Path
import nflDataFunctions as nfldf

pd.set_option('display.max_rows', 100)

In [7]:
games, players, plays, pffData = nfldf.loadData()
weekData = nfldf.loadWeek()

In [11]:
plays.iloc[7:15]

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,...,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI
7,2018090600,402,(9:08) M.Ryan pass incomplete deep left to M.S...,1,1,10,ATL,play_type_pass,PHI,44,...,0.0,09:08:00,54.0,NaN,NaN,I,0,0,-0.429863,False
8,2018090600,492,(7:01) M.Ryan pass short left to T.Coleman pus...,1,2,13,ATL,play_type_pass,PHI,36,...,0.0,07:01:00,46.0,NaN,NaN,C,26,26,1.879804,False
9,2018090600,521,(6:19) M.Ryan pass short left to A.Hooper to P...,1,1,10,ATL,play_type_pass,PHI,10,...,0.0,06:19:00,20.0,NaN,NaN,C,3,3,0.045665,False
10,2018090600,545,(5:38) M.Ryan pass incomplete short left.,1,2,7,ATL,play_type_pass,PHI,7,...,0.0,05:38:00,17.0,NaN,NaN,I,0,0,-0.360976,False
11,2018090600,636,(5:07) (Shotgun) M.Ryan pass incomplete short ...,1,3,3,ATL,play_type_pass,PHI,3,...,0.0,05:07:00,13.0,NaN,NaN,I,0,0,-0.874640,False
12,2018090600,752,(3:46) (Shotgun) N.Foles pass short left to N....,1,2,4,PHI,play_type_pass,PHI,37,...,0.0,03:46:00,47.0,NaN,NaN,C,6,6,0.895417,False
13,2018090600,776,(3:06) (Shotgun) N.Foles sacked at PHI 36 for ...,1,1,10,PHI,play_type_sack,PHI,43,...,0.0,03:06:00,53.0,NaN,NaN,S,-7,-7,-0.980175,False
14,2018090600,839,(1:21) (Shotgun) N.Foles FUMBLES (Aborted) at ...,1,3,12,PHI,play_type_sack,PHI,41,...,0.0,01:21:00,51.0,NaN,NaN,S,-6,-6,-2.092108,False


In [20]:
plays.iloc[8]['playDescription']

'(7:01) M.Ryan pass short left to T.Coleman pushed ob at PHI 10 for 26 yards (R.McLeod).'

In [19]:
dfplay['x'].iloc[-2] - dfplay['x'].iloc[0]

-26.54

In [28]:
players

,nflId,height,weight,birthDate,collegeName,position,displayName
0,2539334,72,190,1990-09-10,Washington,CB,Desmond Trufant
1,2539653,70,186,1988-11-01,Southeastern Louisiana,CB,Robert Alford
2,2543850,69,186,1991-12-18,Purdue,SS,Ricardo Allen
3,2555162,73,227,1994-11-04,Louisiana State,MLB,Deion Jones
4,2555255,75,232,1993-07-01,Minnesota,OLB,De'Vondre Campbell
...,...,...,...,...,...,...,...
1298,2553798,6-2,235,09/17/1991,Vanderbilt,LS,Andrew East
1299,2556814,6-2,200,05/17/1994,Duke,WR,Max McCaffrey
1300,2552576,78,231,1992-04-25,Oregon State,QB,Sean Mannion
1301,2561469,69,187,1993-03-15,Alabama-Birmingham,CB,Darious Williams


In [31]:
np.where(['Coleman' in name for name in players['displayName']])[0]

array([  22,  284,  556,  806,  871, 1156])

In [34]:
players[['displayName','position','nflId']].iloc[22]

displayName    Tevin Coleman
position                  RB
nflId                2552453
Name: 22, dtype: object

In [36]:
dfplay = weekData[0][(weekData[0]['gameId']==2018090600) & (weekData[0]['playId']==492) & (weekData[0]['team']=='football')]#(weekData[0]['nflId']==46206)]# & (weekData[0]['nflId']=='football')]
dfplay

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route
6133,2018-09-07T01:24:28.799Z,46.01,23.90,0.00,0.00,0.00,NaN,NaN,None,NaN,Football,NaN,NaN,1,football,2018090600,492,left,NaN
6147,2018-09-07T01:24:28.900Z,46.01,23.89,0.00,0.00,0.00,NaN,NaN,None,NaN,Football,NaN,NaN,2,football,2018090600,492,left,NaN
6161,2018-09-07T01:24:29.000Z,46.01,23.89,0.00,0.00,0.00,NaN,NaN,None,NaN,Football,NaN,NaN,3,football,2018090600,492,left,NaN
6175,2018-09-07T01:24:29.099Z,46.01,23.89,0.00,0.00,0.00,NaN,NaN,None,NaN,Football,NaN,NaN,4,football,2018090600,492,left,NaN
6189,2018-09-07T01:24:29.200Z,46.01,23.89,0.00,0.00,0.00,NaN,NaN,man_in_motion,NaN,Football,NaN,NaN,5,football,2018090600,492,left,NaN
6203,2018-09-07T01:24:29.299Z,46.01,23.89,0.00,0.00,0.00,NaN,NaN,None,NaN,Football,NaN,NaN,6,football,2018090600,492,left,NaN
6217,2018-09-07T01:24:29.400Z,46.01,23.89,0.00,0.00,0.00,NaN,NaN,None,NaN,Football,NaN,NaN,7,football,2018090600,492,left,NaN
6231,2018-09-07T01:24:29.500Z,46.01,23.89,0.00,0.00,0.00,NaN,NaN,None,NaN,Football,NaN,NaN,8,football,2018090600,492,left,NaN
6245,2018-09-07T01:24:29.599Z,46.01,23.89,0.00,0.00,0.00,NaN,NaN,None,NaN,Football,NaN,NaN,9,football,2018090600,492,left,NaN
6259,2018-09-07T01:24:29.700Z,46.01,23.89,0.00,0.00,0.00,NaN,NaN,None,NaN,Football,NaN,NaN,10,football,2018090600,492,left,NaN


In [132]:
dfplay = weekData[0][(weekData[0]['gameId']==2021090900) & (weekData[0]['playId']==187) & (weekData[0]['nflId']==46206)]# & (weekData[0]['nflId']=='football')]
dfplay

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event
2243,2021090900,187,46206.0,1,2021-09-10T00:29:15.000,86.0,DAL,left,77.59,31.25,0.11,0.42,0.01,304.27,215.68,line_set
2244,2021090900,187,46206.0,2,2021-09-10T00:29:15.100,86.0,DAL,left,77.58,31.25,0.08,0.39,0.01,307.08,235.66,None
2245,2021090900,187,46206.0,3,2021-09-10T00:29:15.200,86.0,DAL,left,77.57,31.25,0.15,0.49,0.01,308.03,277.04,None
2246,2021090900,187,46206.0,4,2021-09-10T00:29:15.300,86.0,DAL,left,77.55,31.25,0.22,0.45,0.02,310.11,277.16,None
2247,2021090900,187,46206.0,5,2021-09-10T00:29:15.400,86.0,DAL,left,77.51,31.24,0.45,0.83,0.04,301.52,270.42,None
2248,2021090900,187,46206.0,6,2021-09-10T00:29:15.500,86.0,DAL,left,77.43,31.24,0.73,1.29,0.08,288.09,267.12,ball_snap
2249,2021090900,187,46206.0,7,2021-09-10T00:29:15.600,86.0,DAL,left,77.31,31.21,1.17,1.89,0.13,286.69,260.18,None
2250,2021090900,187,46206.0,8,2021-09-10T00:29:15.700,86.0,DAL,left,77.16,31.19,1.48,2.06,0.15,274.39,260.41,None
2251,2021090900,187,46206.0,9,2021-09-10T00:29:15.800,86.0,DAL,left,76.97,31.15,1.92,2.42,0.19,269.12,258.78,None
2252,2021090900,187,46206.0,10,2021-09-10T00:29:15.900,86.0,DAL,left,76.74,31.10,2.38,3.02,0.24,263.80,257.89,None
